In [1]:
import sys
sys.path.append("..")

In [2]:
import numpy as np
import pandas as pd

from re import sub
from pprint import pprint
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
from nltk import ngrams

from src.utils.tag_topic import TopicPriorityTag
from src.utils.clean_comments import CommentCleaner

import gspread as gs
import os

In [3]:
df = pd.read_csv('../output/all_comments.csv')
df.head()

,comment_id,date_aired,sender_name,text,clean_text,clean_text_word_count,text_word_count,gender
0,202207070061,2022-07-07,John ***********,Palala lamang po sa mga ofws sa ksa wag mameke...,palala ofws ksa wag mameke dokyumento nahuli n...,8,17,masculine
1,202207070065,2022-07-07,Agripina ***********,Hi Sir Arnel good evening Po!watching from San...,joaquin iloilo yrs dole akap financial assista...,8,22,feminine
2,202207070066,2022-07-07,Merybeth ***********,Hello sir and ma'am Sana matulungan nyo nman a...,matulungan 14year abroad pandemic hindi balik ...,12,32,feminine
3,202207070067,2022-07-07,Airam ***********,"Sir arnel and all the staff, Good evening po! ...",staff ofw kuwait years member owwa lockdown 20...,13,37,feminine
4,202207070068,2022-07-07,Karen ***********,Paano Kong Hindi Naman distress gusto na lng t...,hindi distress gusto 22yrs lebanon active owwa...,12,35,feminine


### Detect topic per comment using previous results

In [11]:
tagger = TopicPriorityTag()

In [13]:
#test on longest string
field_length = df['clean_text'].map(len)
test = df['clean_text'].iloc[1]#loc[field_length.argmax()]
#test = 'Sir gusto po sana namin maka avail ng balik Pinas trabho kaso po wla po certification letter sa company ng mister ko na nagppatunay na hindi po sya nkabalik sa work nya dahil sa covid..paano po pwede gawin sir salamat po.kc parang malabo makakuha kasi dito na sya sa Pinas.salamat po'
tagger.tag_topic_category(test)

['Repatriation']

In [14]:
df['topic_category'] = df['clean_text'].apply(lambda x: tagger.tag_topic_category(x))
df['priority']= df['topic_category'].apply(lambda x: tagger.tag_topic_priority(x))
#df['topic_category'] = df['topic_category'].apply(lambda x: '; '.join(x))
df

,comment_id,date_aired,sender_name,text,clean_text,clean_text_word_count,text_word_count,gender,topic_category,priority
0,202207070061,2022-07-07,John ***********,Palala lamang po sa mga ofws sa ksa wag mameke...,palala ofws ksa wag mameke dokyumento nahuli n...,8,17,masculine,[],0
1,202207070065,2022-07-07,Agripina ***********,Hi Sir Arnel good evening Po!watching from San...,joaquin iloilo yrs dole akap financial assista...,8,22,feminine,[Repatriation],2
2,202207070066,2022-07-07,Merybeth ***********,Hello sir and ma'am Sana matulungan nyo nman a...,matulungan 14year abroad pandemic hindi balik ...,12,32,feminine,"[Calamity, Disability and Death Assistance, Me...",3
3,202207070067,2022-07-07,Airam ***********,"Sir arnel and all the staff, Good evening po! ...",staff ofw kuwait years member owwa lockdown 20...,13,37,feminine,"[Calamity, Disability and Death Assistance, Me...",3
4,202207070068,2022-07-07,Karen ***********,Paano Kong Hindi Naman distress gusto na lng t...,hindi distress gusto 22yrs lebanon active owwa...,12,35,feminine,"[Membership, Certifications and Online Services]",1
...,...,...,...,...,...,...,...,...,...,...
17531,202207211605,2022-07-21,Sarupac ***********,Magandang araw po saan po pwede lumapit kung m...,araw lumapit repatriation reuest agency oman h...,19,57,unknown,"[Livelihood Assistance, Repatriation]",2
17532,202207211606,2022-07-21,Sarupac ***********,Magandang araw po saan po pwede lumapit kung m...,araw lumapit repatriation reuest agency oman h...,21,51,unknown,"[Livelihood Assistance, Repatriation]",2
17533,202207211607,2022-07-21,Muslimah ***********,From U.A.E pano po mag aply s owwa scholarship,aply owwa scholarship,3,9,feminine,[Scholarship],2
17534,202207211608,2022-07-21,Lina ***********,Sir tanong lang nagrenew kasi kami ng mga kasa...,tanong nagrenew kasama polo owwa araw bago pap...,12,27,feminine,[],0


In [23]:
df['gender'].value_counts(normalize=True)

feminine            0.488880
unknown             0.308052
masculine           0.143875
androgynous         0.026004
mostly_feminine     0.018077
mostly_masculine    0.015112
Name: gender, dtype: float64

In [20]:
len(df[df['topic_category'].map(len)>0])

8596

In [209]:
df.to_csv('../output/all_comments_topic_tagged.csv')

### Count topic category per guessed gender

In [15]:
gender_df = df[df['topic_category'].map(len)>0][['gender','topic_category']].copy()
gender_df = gender_df.explode('topic_category')
gender_df['gender'] = gender_df['gender'].apply(lambda x: sub('mostly_','',x))
gender_df

,gender,topic_category
1,feminine,Repatriation
2,feminine,"Calamity, Disability and Death Assistance"
2,feminine,"Membership, Certifications and Online Services"
2,feminine,Repatriation
3,feminine,"Calamity, Disability and Death Assistance"
...,...,...
17531,unknown,Repatriation
17532,unknown,Livelihood Assistance
17532,unknown,Repatriation
17533,feminine,Scholarship


In [16]:
pgdf = gender_df.groupby(['topic_category','gender']).size().to_frame('count').reset_index()
pgdf = pgdf.pivot(index='topic_category',columns='gender', values='count')
pgdf

gender,androgynous,feminine,masculine,unknown
topic_category,,,,
"Calamity, Disability and Death Assistance",19,339,94,220
"Detention, Abuse and Harassment",1,20,4,28
Health,14,276,103,142
Livelihood Assistance,6,264,122,187
"Membership, Certifications and Online Services",43,827,204,528
Reintegration,4,182,72,135
Repatriation,110,2065,752,1346
Scholarship,84,1287,304,701
Training,3,35,22,27


In [21]:
pgdf.to_csv('../output/topic_category_per_gender.csv')